### 오디오 다운로드

In [ ]:
from predict_stock_outlook import extract_video_audio

df = pd.read_csv("data_kr/video/자료 수집 최종본.csv")

for row in df.itertuples():
    if pd.isna(row.url) or row.url == '' or row.category != "video":
        continue

    code = str(row.code).zfill(6)
    audio_dir = f'data_kr/video/audio/{row.sector}/{code}/'
    text_dir = f'data_kr/video/text/{row.sector}/{code}/'
    os.makedirs(audio_dir, exist_ok=True)

    if audio_dir + f'{row.year}-{row.quarter}' != 'data_kr/video/audio/산업재/003490/2016-Q2':
        continue
    
    if extract_video_audio("link", row.url, audio_dir + f'{row.year}-{row.quarter}'):
        with open('data_kr/video/log.txt', "a", encoding="utf-8") as log_file:
            timestamp = datetime.now().strftime("[%Y-%m-%d %H:%M:%S]")
            log_file.write(f"{timestamp} audio download completed: {audio_dir + f'{row.year}-{row.quarter}'}\n")
    else:
        with open('data_kr/video/log.txt', "a", encoding="utf-8") as log_file:
            timestamp = datetime.now().strftime("[%Y-%m-%d %H:%M:%S]")
            log_file.write(f"{timestamp} audio download error: {audio_dir + f'{row.year}-{row.quarter}'}\n")

### 텍스트로 변환

In [ ]:
from predict_stock_outlook import audio2text

df = pd.read_csv("data_kr/video/자료 수집 최종본.csv")

for row in df.itertuples():
    if pd.isna(row.url) or row.url == '' or row.category != "video":
        continue

    code = str(row.code).zfill(6)
    audio_dir = f'data_kr/video/audio/{row.sector}/{code}/'
    text_dir = f'data_kr/video/text/{row.sector}/{code}/'
    os.makedirs(text_dir, exist_ok=True)
    
    if audio_dir + f'{row.year}-{row.quarter}' != 'data_kr/video/audio/산업재/003490/2016-Q2':
        continue
    
    try:
        text = audio2text(audio_dir + f'{row.year}-{row.quarter}')
        with open(text_dir + f'{row.year}-{row.quarter}.txt', "w", encoding="utf-8") as f:
            f.write(text)
        with open('data_kr/video/log.txt', "a", encoding="utf-8") as log_file:
            timestamp = datetime.now().strftime("[%Y-%m-%d %H:%M:%S]")
            log_file.write(f"{timestamp} whisper completed: {text_dir + f'{row.year}-{row.quarter}'}\n")
    except Exception as e:
        with open('data_kr/video/log.txt', "a", encoding="utf-8") as log_file:
            timestamp = datetime.now().strftime("[%Y-%m-%d %H:%M:%S]")
            log_file.write(f"{timestamp} whisper error: {text_dir + f'{row.year}-{row.quarter}'}\n")

### 텍스트 token 수 확인

In [ ]:
df = pd.read_csv("data_kr/video/자료 수집 최종본.csv")

import tiktoken
# 예: GPT-4용 인코더 불러오기
encoding = tiktoken.encoding_for_model("gpt-4o")
total_tokens = 0

for row in tqdm(df.itertuples(), total=len(df), desc="checking tokens"):
	if pd.isna(row.url) or row.url == '':
		continue

	code = str(row.code).zfill(6)
	text_dir = f'data_kr/video/text/{row.sector}/{code}/'
	os.makedirs(text_dir, exist_ok=True)
    
	try:
		filename = f'{row.year}-{row.quarter}-{str(row.month).zfill(2)}-{row.week}.txt'
		with open(text_dir + filename, "r", encoding="utf-8") as file:
			text = file.read()
			system_prompt = """
		너는 경제 전문 뉴스 분석 AI야. 사용자가 지정한 종목(회사명)과 직접적으로 관련된 정보만 선택해 핵심적으로 요약해.
		사실 기반으로 요약하고, 감성이나 추론이 필요한 경우에는 중립적으로 표현해.
		"""

			user_prompt = f"""
		다음은 경제 뉴스 기사입니다.

		이 기사에서 **한국 상장 기업 "{code}"**과 관련된 내용만 골라 요약해 주세요.

		요약 기준:
		- "{code}"이 언급된 부분 중심
		- 관련 사업, 실적, 주가, 시장 반응, 경쟁사와의 연관성
		- 정부 정책, 산업 트렌드 등 외부 요인 중 관련 있는 부분
		- 부정적/긍정적 논조도 간단히 언급 (있는 경우)

		형식은 간결한 문장 또는 Bullet Point 형식으로 작성해 주세요.

		기사 전문:
		{text}
		"""
			total_prompt = system_prompt + user_prompt
			tokens = encoding.encode(total_prompt)
			total_tokens += len(tokens)
		with open('data_kr/video/num_token.txt', "a", encoding="utf-8") as log_file:
			timestamp = datetime.now().strftime("[%Y-%m-%d %H:%M:%S]")
			log_file.write(f"{text_dir}{filename} len: {len(text)}, token: {len(tokens)}\n")
	except Exception as e:
		with open('data_kr/video/num_token.txt', "a", encoding="utf-8") as log_file:
			timestamp = datetime.now().strftime("[%Y-%m-%d %H:%M:%S]")
			log_file.write(f"{text_dir}{filename} error: " + e + "\n")

with open('data_kr/video/num_token.txt', "a", encoding="utf-8") as log_file:
	log_file.write(f"total tokens: {total_tokens}\n")

### LLM으로 요약

In [ ]:
from predict_stock_outlook import summarize_text

df = pd.read_csv("data_kr/video/자료 수집 최종본.csv")

for row in tqdm(df.itertuples(), total=len(df), desc="LLM summarizing"):
    if pd.isna(row.url) or row.url == '':
        continue

    code = str(row.code).zfill(6)
    name = row.name
    text_dir = f'data_kr/video/text/{row.sector}/{code}/'
    summary_dir = f'preprocessed_data/llm/summary/{row.sector}/{code}/'
    os.makedirs(summary_dir, exist_ok=True)
        
    try:
        filename = f'{row.year}-{row.quarter}-{str(row.month).zfill(2)}-{row.week}.txt'
        stock = f'{name}({code})'
        with open(text_dir + filename, "r", encoding="utf-8") as file:
            text = file.read()
        summary = summarize_text(text, stock)
        with open(summary_dir + filename, "w", encoding="utf-8") as f:
            f.write(summary)
        with open('preprocessed_data/llm/summary/log.txt', "a", encoding="utf-8") as log_file:
            timestamp = datetime.now().strftime("[%Y-%m-%d %H:%M:%S]")
            log_file.write(f"{timestamp} summary completed: {summary_dir + filename}\n")
    except Exception as e:
        with open('preprocessed_data/llm/summary/log.txt', "a", encoding="utf-8") as log_file:
            timestamp = datetime.now().strftime("[%Y-%m-%d %H:%M:%S]")
            log_file.write(f"{timestamp} summary error: {summary_dir + filename}\t error: {e}\n")

### LLM으로 기사 자막요약을 통해 등락 예측

In [ ]:
from predict_stock_outlook import predict_market_from_summary
import os
from tqdm import tqdm
from datetime import datetime
import pandas as pd

df = pd.read_csv('data_kr/video/뉴스 기사 수집본.csv', encoding='utf-8')

for code in df["code"].unique():
    df_ = df[df["code"] == code].reset_index(drop=True)

    for row in tqdm(df_.itertuples(), total=len(df_), desc=f"{code}LLM predicting"):
        if pd.isna(row.url) or row.url == '':
            continue
        
        code = str(row.code).zfill(6)	
        name = row.name
        article_dir = f'data_kr/video/text/{row.sector}/{code}/'
        predict_dir = f'preprocessed_data/llm/predict_text/{row.sector}/{code}/'
        os.makedirs(predict_dir, exist_ok=True)

        try:
            filename = f'{row.year}-{row.quarter}-{str(row.month).zfill(2)}-{row.week}.txt'
            stock = f'{name}({code})'
            with open(f'{article_dir}{filename}', "r", encoding="utf-8") as file:
                article = file.read()
            data = predict_market_from_summary(article, f'{name}({code})')
            
            with open(f'{predict_dir}{filename}', "w", encoding="utf-8") as file:
                file.write(data)
            
            with open('preprocessed_data/llm/predict_text/log.txt', "a", encoding="utf-8") as log_file:
                timestamp = datetime.now().strftime("[%Y-%m-%d %H:%M:%S]")
                log_file.write(f"{timestamp} predict completed: {predict_dir}{filename}\n")
        except Exception as e:
            with open('preprocessed_data/llm/predict_text/log.txt', "a", encoding="utf-8") as log_file:
                timestamp = datetime.now().strftime("[%Y-%m-%d %H:%M:%S]")
                log_file.write(f"{timestamp} predict error: {predict_dir}{filename}\n")  

### LLM으로 영상 자막요약을 통해 등락 예측

In [ ]:
from predict_stock_outlook import predict_market_from_summary
import os
from tqdm import tqdm
from datetime import datetime
import pandas as pd

df = pd.read_csv('data_kr/video/뉴스 영상 수집본.csv', encoding='utf-8')

for code in df["code"].unique():
    df_ = df[df["code"] == code].reset_index(drop=True)

    for row in tqdm(df_.itertuples(), total=len(df_), desc=f"{code}LLM predicting"):
        if pd.isna(row.url) or row.url == '':
            continue
        
        code = str(row.code).zfill(6)	
        name = row.name
        script_dir = f'data_kr/video/script/{row.sector}/{code}/'
        predict_dir = f'preprocessed_data/llm/predict_video/{row.sector}/{code}/'
        os.makedirs(predict_dir, exist_ok=True)

        try:
            filename = f'{row.year}-{row.quarter}-{str(row.month).zfill(2)}-{row.week}.txt'
            stock = f'{name}({code})'
            with open(f'{script_dir}{filename}', "r", encoding="utf-8") as file:
                script = file.read()
            prediction = predict_market_from_summary(script, f'{name}({code})')
            
            with open(f'{predict_dir}{filename}', "w", encoding="utf-8") as file:
                file.write(prediction)
            
            with open('preprocessed_data/llm/predict_video/log.txt', "a", encoding="utf-8") as log_file:
                timestamp = datetime.now().strftime("[%Y-%m-%d %H:%M:%S]")
                log_file.write(f"{timestamp} predict completed: {predict_dir}{filename}\n")
        except Exception as e:
            with open('preprocessed_data/llm/predict_video/log.txt', "a", encoding="utf-8") as log_file:
                timestamp = datetime.now().strftime("[%Y-%m-%d %H:%M:%S]")
                log_file.write(f"{timestamp} predict error: {predict_dir}{filename}\n")  

### LLM으로 영상자막요약 + 기사자막요약을 통해 등락 예측

In [26]:
from predict_stock_outlook import predict_market_from_summary

df_v = pd.read_csv('data_kr/video/뉴스 영상 수집본.csv', encoding='utf-8')
df_v.rename(columns={'url':'v_url', 'upload_dt':'v_upload_dt'}, inplace=True)
df_v = df_v[["year","quarter","month","week","code","name","sector","after","before","v_url","v_upload_dt"]]

df_a = pd.read_csv('data_kr/video/뉴스 기사 수집본.csv', encoding='utf-8')
df_a.rename(columns={'url':'a_url', 'upload_dt':'a_upload_dt'}, inplace=True)
df_a = df_a[["a_url","a_upload_dt"]]

df = pd.concat([df_v, df_a], axis=1)

for code in df["code"].unique():
    df_ = df[df["code"] == code].reset_index(drop=True)
    
    predict_list = []
    reason_list = []

    for row in tqdm(df_.itertuples(), total=len(df_), desc=f"{code}LLM predicting"):
        if pd.isna(row.v_url) or pd.isna(row.a_url) :
            predict_list.append(None)
            reason_list.append(None)
            continue
        
        code = str(row.code).zfill(6)
        name = row.name
        v_summary_dir = f'preprocessed_data/llm/summary_video/{row.sector}/{code}/'
        t_summary_dir = f'preprocessed_data/llm/summary_text/{row.sector}/{code}/'
        predict_dir = f'preprocessed_data/llm/predict_mix/{row.sector}/{code}/'
        os.makedirs(predict_dir, exist_ok=True)

        try:
            filename = f'{row.year}-{row.quarter}-{str(row.month).zfill(2)}-{row.week}.txt'
            stock = f'{name}({code})'
            with open(f'{v_summary_dir}{filename}', "r", encoding="utf-8") as file:
                v_summary = file.read()
            with open(f'{t_summary_dir}{filename}', "r", encoding="utf-8") as file:
                t_summary = file.read()
 
            data = predict_market_from_summary(v_summary + t_summary, f'{name}({code})')
            
            with open(f'{predict_dir}{filename}', "w", encoding="utf-8") as file:
                file.write(data)
 
            predict = data.split('\n')[0].split(':')[1].strip()
            reason = data.split('\n')[1].split(':')[1].strip()
            predict_list.append(predict)
            reason_list.append(reason)
            
            with open('preprocessed_data/llm/predict_mix/log.txt', "a", encoding="utf-8") as log_file:
                timestamp = datetime.now().strftime("[%Y-%m-%d %H:%M:%S]")
                log_file.write(f"{timestamp} predict completed: {predict_dir}{filename}\n")
        except Exception as e:
            with open('preprocessed_data/llm/predict_mix/log.txt', "a", encoding="utf-8") as log_file:
                timestamp = datetime.now().strftime("[%Y-%m-%d %H:%M:%S]")
                log_file.write(f"{timestamp} predict error: {predict_dir}{filename}\n")

120LLM predicting:   1%|▋                                                                    | 2/204 [00:05<09:18,  2.76s/it]


KeyboardInterrupt: 

### 기사 예측 결과 정리

In [ ]:
df=pd.read_csv('./data_kr/video/뉴스 기사 수집본.csv', encoding='utf-8')

for code in df["code"].unique():
    df_ = df[df["code"] == code].reset_index(drop=True)
    predict_list = []
    reason_list = []
    score_list = []

    for row in tqdm(df_.itertuples(), total=len(df_), desc=f"{code}LLM predicting"):
        if pd.isna(row.url) or row.url == '':
            predict_list.append(None)
            reason_list.append(None)
            score_list.append(None)
            continue
        
        code = str(row.code).zfill(6)	
        name = row.name
        predict_dir = f'preprocessed_data/llm/predict_text/{row.sector}/{code}/'
        os.makedirs(predict_dir, exist_ok=True)

        try:
            filename = f'{row.year}-{row.quarter}-{str(row.month).zfill(2)}-{row.week}.txt'
            stock = f'{name}({code})'
            with open(f'{predict_dir}{filename}', "r", encoding="utf-8") as file:
                data = file.read()
                
            predict = data.split('\n')[0].split(':')[1].strip()
            reason = data.split('\n')[1].split(':')[1].strip()
            score = data.split('\n')[2].split(':')[1].strip()
            predict_list.append(predict)
            reason_list.append(reason)
            score_list.append(int(score))
            
        except Exception as e:
            predict_list.append("중립")
            reason_list.append("관련 없음")
            score_list.append(0)
            
    df_predict = df_.copy()
    df_predict["prediction"] = predict_list
    df_predict["reason"] = reason_list
    df_predict["score"] = score_list
    df_predict = df_predict[["year", "quarter", "month", "week", "code", "name", "sector", "upload_dt", "prediction", "reason", "score"]]
    df_predict.to_csv(f"{predict_dir}{code}.csv", index=False, encoding="utf-8")

### 영상 예측 결과 정리

In [ ]:
df=pd.read_csv('./data_kr/video/뉴스 영상 수집본.csv', encoding='utf-8')
for code in df["code"].unique():
    df_ = df[df["code"] == code].reset_index(drop=True)
    predict_list = []
    reason_list = []
    score_list = []

    for row in tqdm(df_.itertuples(), total=len(df_), desc=f"{code}LLM predicting"):
        if pd.isna(row.url) or row.url == '':
            predict_list.append(None)
            reason_list.append(None)
            score_list.append(None)
            continue
        
        code = str(row.code).zfill(6)	
        name = row.name
        predict_dir = f'preprocessed_data/llm/predict_video/{row.sector}/{code}/'
        os.makedirs(predict_dir, exist_ok=True)

        try:
            filename = f'{row.year}-{row.quarter}-{str(row.month).zfill(2)}-{row.week}.txt'
            stock = f'{name}({code})'
            with open(f'{predict_dir}{filename}', "r", encoding="utf-8") as file:
                data = file.read()
                
            predict = data.split('\n')[0].split(':')[1].strip()
            reason = data.split('\n')[1].split(':')[1].strip()
            score = data.split('\n')[2].split(':')[1].strip()
            predict_list.append(predict)
            reason_list.append(reason)
            score_list.append(int(score))
            
        except Exception as e:
            predict_list.append("불가능")
            reason_list.append("관련 없음")
            score_list.append(0)
            
    df_predict = df_.copy()
    df_predict["prediction"] = predict_list
    df_predict["reason"] = reason_list
    df_predict["score"] = score_list
    df_predict = df_predict[["year", "quarter", "month", "week", "code", "name", "sector", "upload_dt", "prediction", "reason", "score"]]
    df_predict.to_csv(f"{predict_dir}{code}.csv", index=False, encoding="utf-8")